## Проверка загруженной модели

In [ ]:
APP_NAME='Flights_Spark_TrainValidationSplit'

In [ ]:
S3_DATA='s3a://mlflow-test/data/flights-larger.csv'

### Используйте свой путь до директории с моделью

In [ ]:
saved_model_path = 'Student_Name_flights_LR_only'

In [1]:
#! pip install findspark

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession

In [10]:
#! pip install mlflow

In [3]:
import mlflow

In [4]:
from datetime import datetime

In [2]:
!ls $HOME/.aws

config	credentials


In [6]:
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .config("spark.jars", "mlflow-spark-1.27.0.jar")\
    .getOrCreate()

In [7]:
# spark.stop()

In [8]:
data = (spark.read.format('csv')
        .options(header='true', inferSchema='true')
        .load(S3_DATA))

In [9]:
assembler = VectorAssembler(inputCols=["mile"], outputCol="features")
out_data = assembler.transform(data)

In [10]:
out_data.show()

+---+---+---+-------+------+---+----+------+--------+-----+--------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|features|
+---+---+---+-------+------+---+----+------+--------+-----+--------+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27| [157.0]|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102|   NA| [466.0]|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19| [738.0]|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|[2248.0]|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22| [386.0]|
|  3| 28|  1|     B6|   377|LGA|1076| 13.33|     182|   70|[1076.0]|
|  5| 28|  6|     B6|   904|ORD| 740|  9.58|     130|   47| [740.0]|
|  1| 19|  2|     UA|   820|SFO| 679| 12.75|     123|  135| [679.0]|
|  8|  5|  5|     US|  2175|LGA| 214|  13.0|      71|  -10| [214.0]|
|  5| 27|  5|     AA|  1240|ORD|1197| 14.42|     195|  -11|[1197.0]|
|  8| 20|  6|     B6|   119|JFK|1182| 14.67|     198|   20|[1182.0]|
|  2|  3|  1|     AA|  1881|JFK|10

In [11]:
evaluator = RegressionEvaluator(labelCol='duration')

In [13]:
loaded_model = mlflow.spark.load_model(saved_model_path)

2022/07/27 13:13:47 INFO mlflow.spark: File 'Student_Name_flights_LR_only/sparkml' not found on DFS. Will attempt to upload the file.
2022/07/27 13:13:48 INFO mlflow.spark: Copied SparkML model to /tmp/mlflow/ac2d15c7-5636-46a9-afc0-39de128d968f


In [17]:
predictions = loaded_model.transform(out_data)\
    .select("features", "duration", "prediction")

In [18]:
predictions.show()

+--------+--------+------------------+
|features|duration|        prediction|
+--------+--------+------------------+
| [157.0]|      51|63.381402677702845|
| [466.0]|     102|101.03866245585063|
| [738.0]|     127|134.18680051299043|
|[2248.0]|     365| 318.2077139919651|
| [386.0]|      85| 91.28921008610362|
|[1076.0]|     182| 175.3782367751715|
| [740.0]|     130| 134.4305368222341|
| [679.0]|     123|  126.996579390302|
| [214.0]|      71| 70.32788749114758|
|[1197.0]|     195|190.12428348441384|
|[1182.0]|     198|188.29626116508626|
|[1090.0]|     200| 177.0843909398772|
|[1028.0]|     193| 169.5285653533233|
| [733.0]|     125|133.57745973988125|
| [733.0]|     129|133.57745973988125|
| [292.0]|     102|  79.8336035516509|
| [612.0]|     109|118.83141303063891|
| [301.0]|      88| 80.93041694324745|
| [414.0]|      91| 94.70151841551508|
|[1846.0]|     275|269.21671583398637|
+--------+--------+------------------+
only showing top 20 rows



In [19]:
evaluator.evaluate(predictions)

17.19136134051717

In [20]:
spark.stop()